In [2]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "TS"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [ ]:
# Load data
well_name = 'xxx'
well = project.get_well(well_name)
well_data = well.data.copy()
well_data.columns = [col.replace('LFP_', '') for col in well_data.columns]
replace_cols = {
    'NEU': 'NPHI',
    'DEN': 'RHOB',
    'NEUT_HC': 'NPHI_HC',
    'DEN_HC': 'RHOB_HC',
    'VSH': 'VSHALE'
}
well_data.rename(columns=replace_cols, inplace=True)
well_data.columns

# Quick PP Interpretation

In [5]:
from quick_pp.lithology.thin_beds import ThinBeds
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix, normalize_volumetric, estimate_shale_porosity_trend
from quick_pp.qaqc import badhole_flagging, handle_outer_limit, neu_den_xplot_hc_correction, den_correction
from quick_pp.saturation import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_typing, estimate_vsh_gr
from quick_pp.plotter.plotter import plotly_log, neutron_density_xplot
from quick_pp.utils import zone_flagging, min_max_line

## Estimate Litholoy

In [6]:
args = {
    'fluid_point': (1, 1),
    'dry_min1_point': (-.02, 2.65),
    'dry_clay_point': (.4, 2.75),
    'dry_sand_poro': 0.38,
    'dry_shale_poro': 0.18,
}
tb_model = ThinBeds(**args)

# filtered_df = well_data[(well_data.DEPTH > 3800) & (well_data.DEPTH < 3900)].copy()
filtered_df = well_data.copy()
filtered_df['DEPTH'] = filtered_df.DEPTH * 3.28084  # Convert to feet
vsand, vshale, phit, vsh_lam, vsh_dis, vsand_dis, phit_sand = tb_model.estimate_litho_poro(filtered_df.NPHI, filtered_df.RHOB)

In [ ]:
from quick_pp.lithology.thin_beds import vsh_phit_xplot

vsh_phit_xplot(vshale, phit, **args)

In [ ]:
neutron_density_xplot(filtered_df['NPHI'], filtered_df['RHOB'], **args)

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(20, 2))
plt.plot(filtered_df.DEPTH.values, vshale, label='VSH', color='brown')
plt.plot(filtered_df.DEPTH.values, vsh_lam, label='VSH_Lam', color='blue')
plt.plot(filtered_df.DEPTH.values, filtered_df.VLAM_TS, label='VSH_Lam_TS', color='orange')
plt.plot(filtered_df.DEPTH.values, vsh_dis, label='VSH_Dis', color='green')

plt.xlabel('Depth')
plt.ylabel('Volume')
plt.ylim(0, 1)
plt.title('Comparison of VSH, VSH_Lam, and VSH_Dis Across Depth')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plotting
plt.figure(figsize=(20, 2))
plt.plot(filtered_df.DEPTH.values, phit, label='PHIT', color='brown')
plt.plot(filtered_df.DEPTH.values, phit_sand, label='PHIT_Sand', color='blue')
plt.plot(filtered_df.DEPTH.values, filtered_df.PHISAND_TS, label='PHIT_Sand_TS', color='red')

plt.xlabel('Depth')
plt.ylabel('Porosity')
plt.ylim(0, 0.5)
plt.title('Comparison of PHIT and PHIT_Sand Across Depth')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plotting
plt.figure(figsize=(20, 2))
plt.plot(filtered_df.DEPTH.values, vsand, label='VSand', color='gold')
plt.plot(filtered_df.DEPTH.values, vsand_dis, label='VSand_Dis', color='orange')

plt.xlabel('Depth')
plt.ylabel('Volume')
plt.ylim(0, 1)
plt.title('Comparison of VSand and VSand_Dis Across Depth')
plt.legend()
plt.grid()
plt.show()

### Implement HC Correction

In [12]:
args.update({
    'hc_corr_angle': 50,
    'hc_buffer': 0.01,
})
vsh_gr = estimate_vsh_gr(filtered_df['GR'], min_gr=0, max_gr=140)
nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
    filtered_df['NPHI'], filtered_df['RHOB'], vsh_gr=vsh_gr,
    dry_min1_point=args['dry_min1_point'],
    dry_clay_point=args['dry_clay_point'],
    corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
)

# Correct density log
rhob_corr = den_correction(nphihc, filtered_df['GR'], vsh_gr=vsh_gr, alpha=0.1)

In [13]:
args = {
    'fluid_point': (1, 1),
    'dry_min1_point': (-.02, 2.65),
    'dry_clay_point': (.4, 2.75),
    'dry_sand_poro': 0.38,
    'dry_shale_poro': 0.18,
}
tb_model = ThinBeds(**args)

vsand, vshale, phit, vsh_lam, vsh_dis, vsand_dis, phit_sand = tb_model.estimate_litho_poro(nphihc, rhob_corr)
filtered_df['VSAND'] = vsand
filtered_df['VSH'] = vshale
filtered_df['PHIT'] = phit
filtered_df['VSH_LAM'] = vsh_lam
filtered_df['VSH_DIS'] = vsh_dis
filtered_df['VSAND_DIS'] = vsand_dis
filtered_df['PHISAND'] = phit_sand

In [ ]:
from quick_pp.lithology.thin_beds import vsh_phit_xplot

vsh_phit_xplot(vshale, phit, **args)

In [ ]:
neutron_density_xplot(nphihc, rhobhc, **args)

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(20, 2))
plt.plot(filtered_df.DEPTH.values, vshale, label='VSHale', color='brown')
plt.plot(filtered_df.DEPTH.values, vsh_lam, label='VSH_Lam', color='blue')
plt.plot(filtered_df.DEPTH.values, filtered_df.VLAM_TS, label='VSH_Lam_TS', color='orange')
plt.plot(filtered_df.DEPTH.values, vsh_dis, label='VSH_Dis', color='green')

plt.xlabel('Depth')
plt.ylabel('Volume')
plt.ylim(0, 1)
plt.title('Comparison of VSHale, VSH_Lam, and VSH_Dis Across Depth')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plotting
plt.figure(figsize=(20, 2))
plt.plot(filtered_df.DEPTH.values, phit, label='PHIT', color='brown')
plt.plot(filtered_df.DEPTH.values, phit_sand, label='PHIT_Sand', color='blue')
plt.plot(filtered_df.DEPTH.values, filtered_df.PHISAND_TS, label='PHIT_Sand_TS', color='red')

plt.xlabel('Depth')
plt.ylabel('Porosity')
plt.ylim(0, 0.5)
plt.title('Comparison of PHIT and PHIT_Sand Across Depth')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plotting
plt.figure(figsize=(20, 2))
plt.plot(filtered_df.DEPTH.values, vsand, label='VSand', color='gold')
plt.plot(filtered_df.DEPTH.values, vsand_dis, label='VSand_Dis', color='orange')

plt.xlabel('Depth')
plt.ylabel('Volume')
plt.ylim(0, 1)
plt.title('Comparison of VSand and VSand_Dis Across Depth')
plt.legend()
plt.grid()
plt.show()

## Resistivity Modelling

In [ ]:
filtered_df.columns

In [ ]:
rapparent = tb_model.apparent_resistivity(rv=filtered_df.RV, rh=filtered_df.RH, theta=60)
rsand = tb_model.sand_resistivity_macro(
   rv=filtered_df.RV,
   rh=filtered_df.RH,
   rshale=1,
)
rsand_micro = tb_model.sand_resistivity_micro(
   rv=filtered_df.RV,
   rh=filtered_df.RH,
   rv_shale=1.2,
   rh_shale=1,
)
rmodel = tb_model.resistivity_modelling(
    vsh_lam=vsh_lam,
    rsand=filtered_df.RT,
    rh_shale=1,
    rv_shale=1.2,
    theta=5
)
filtered_df['RSAND'] = rsand
filtered_df['RMODEL'] = rmodel

plt.figure(figsize=(20, 2))
plt.plot(filtered_df.DEPTH, rmodel, label='R model', color='blue')
plt.plot(filtered_df.DEPTH, rapparent, label='R apparent', color='orange')
plt.plot(filtered_df.DEPTH, rsand, label='R sand', color='pink')
plt.plot(filtered_df.DEPTH, rsand_micro, label='R sand micro', color='cyan')
plt.plot(filtered_df.DEPTH, filtered_df.RT, label='RT')
plt.plot(filtered_df.DEPTH, filtered_df.RH, label='RH', color='green')
plt.plot(filtered_df.DEPTH, filtered_df.RV, label='RV', color='purple')
plt.plot(filtered_df.DEPTH, filtered_df.RSAND, label='RSAND', color='red')


plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.yscale('log')
plt.ylim(0.2, 200)


## Water Saturation

In [ ]:
# Debug water saturation
water_salinity = 15000
m = 2
rw = .1
index = filtered_df.DEPTH.values

phit = filtered_df['PHISAND'].fillna(-9999)
rt = filtered_df['RSAND'].fillna(-9999)
vshale = filtered_df['VSH_LAM'].fillna(-9999)
temp_grad = estimate_temperature_gradient(filtered_df['DEPTH'], 'metric')
qv = estimate_qv(vshale, phit, cec_clay=.1)
swt = waxman_smits_saturation(rt, rw, phit, qv)
filtered_df['SW_TS'] = swt

vshale = filtered_df['VSH'].fillna(-9999)
phit = filtered_df['PHIT'].fillna(-9999)
rt = filtered_df['RT'].fillna(-9999)
qv = estimate_qv(vshale, phit, cec_clay=.1)
swt = waxman_smits_saturation(rt, rw, phit, qv)
filtered_df['SWT'] = swt

# Plotting
fig, (ax1, ax3, ax4) = plt.subplots(nrows=3, sharex=True, figsize=(20, 7))

ax1.plot(index, qv, label='Qv')
ax1.set_ylim(0, 5)
ax1.legend()

ax3.plot(index, filtered_df['RT'], label='RT')
ax3.plot(index, filtered_df['RSAND'], label='RSAND')
ax3.set_yscale('log')
ax3.legend()

ax4.plot(index, filtered_df['SWT'], label='SWT')
ax4.plot(index, filtered_df['SW_TS'], label='SW_TS')
ax4.set_ylim(0, 1)
ax4.legend()

fig.tight_layout()

# Plot the results

In [78]:
from collections import OrderedDict
import numpy as np
from quick_pp.plotter.well_log_config import COLOR_DICT

COLOR_DICT['RSAND'] = 'purple'
COLOR_DICT['VSHALE'] = '#A65628'
COLOR_DICT['PHISAND'] = 'purple'
COLOR_DICT['VSH_LAM'] = 'purple'
COLOR_DICT['VSH_DIS'] = 'green'
COLOR_DICT['SW_TS'] = 'purple'

TRACE_DEFS = OrderedDict(
    GR=dict(
        track=1,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': COLOR_DICT['GR']}
    ),
    RT=dict(
        track=2,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['RT']}
    ),
    RHOB=dict(
        track=3,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': COLOR_DICT['RHOB']}
    ),
    PHIT=dict(
        track=4,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': COLOR_DICT['PHIT']}
    ),
    SWT=dict(
        track=5,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['SWT']}
    ),
    VSH=dict(
        track=6,
        secondary_y=False,
        hide_xaxis=False,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tozerox', 'fillpattern_bgcolor': COLOR_DICT['VSHALE'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '-',
            'fillpattern_size': 2, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    NPHI=dict(
        track=3,
        secondary_y=True,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['NPHI']}
    ),
    RSAND=dict(
        track=2,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['RSAND']}
    ),
    VSAND=dict(
        track=6,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tonextx', 'fillpattern_bgcolor': COLOR_DICT['VSAND'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '.',
            'fillpattern_size': 3, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VSH_LAM=dict(
        track=6,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['VSH_LAM']}
    ),
    VSH_DIS=dict(
        track=6,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['VSH_DIS']}
    ),
    PHISAND=dict(
        track=4,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['PHISAND']}
    ),
    SW_TS=dict(
        track=5,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['SW_TS']}
    ),
)

In [79]:
font_size = 8
XAXIS_DEFS = {
    'GR': {
        'title': 'GR',
        'titlefont': {'color': COLOR_DICT['GR'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['GR'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'dtick': 40, 'range': [0, 200], 'type': 'linear', 'zeroline': False
    },
    'RT': {
        'title': 'RT',
        'titlefont': {'color': COLOR_DICT['RT'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['RT'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'range': [np.log10(.2), np.log10(2000)], 'type': 'log',
        'tickmode': 'array', 'tickvals': np.geomspace(0.2, 2000, 5), 'tickangle': -90, 'minor_showgrid': True
    },
    'RHOB': {
        'title': 'RHOB',
        'titlefont': {'color': COLOR_DICT['RHOB'], 'size': font_size},
        'tickformat': ".2f", 'tick0': 1.95, 'dtick': 0.2, 'tickangle': -90,
        'tickfont': {'color': COLOR_DICT['RHOB'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'range': [1.95, 2.95], 'type': 'linear'
    },
    'PHIT': {
        'title': 'PHIT',
        'titlefont': {'color': COLOR_DICT['PHIT'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['PHIT'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85, 'title_standoff': .1,
        'dtick': 0.1, 'range': [0, 0.5], 'type': 'linear', 'zeroline': False
    },
    'PERM': {
        'title': 'PERM',
        'titlefont': {'color': COLOR_DICT['PERM'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['PERM'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .91, 'title_standoff': .1,
        'range': [np.log10(0.1), np.log10(10000)], 'type': 'log', 'tickformat': 'd', 'tickangle': -90,
        'minor_showgrid': True
    },
    'SWT': {
        'title': 'SWT',
        'titlefont': {'color': COLOR_DICT['SWT'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['SWT'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .88, 'title_standoff': .1,
        'dtick': 0.2, 'range': [0, 1], 'type': 'linear', 'zeroline': False
    },
    'NPHI': {
        'title': 'NPHI',
        'titlefont': {'color': COLOR_DICT['NPHI'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['NPHI'], 'size': font_size}, 'zeroline': False,
        'side': 'top', 'anchor': 'free', 'position': .89, 'title_standoff': .1, 'overlaying': 'x3',
        'tickformat': ".2f", 'tick0': -.15, 'dtick': 0.12, 'range': [.45, -.15], 'type': 'linear', 'tickangle': -90
    },
    'RSAND': {
        'title': 'RSAND',
        'titlefont': {'color': COLOR_DICT['RSAND'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['RSAND'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .89,
        'title_standoff': .1, 'range': [np.log10(.2), np.log10(2000)], 'type': 'log', 'overlaying': 'x2',
        'tickmode': 'array', 'tickvals': np.geomspace(0.2, 2000, 5), 'tickangle': -90, 'minor_showgrid': False
    },
    'VSH': {
        'title': 'LITHOLOGY',
        'titlefont': {'color': COLOR_DICT['VSHALE'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['VSHALE'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85, 'title_standoff': .1,
        'dtick': 0.1, 'range': [0, 1], 'type': 'linear', 'zeroline': False
    },
    'VSAND': {
        'title': 'VSAND',
        'titlefont': {'color': COLOR_DICT['VSAND'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['VSAND'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85, 'title_standoff': .1,
        'dtick': 0.1, 'range': [0, 1], 'type': 'linear', 'zeroline': False, 'overlaying': 'x6'
    },
    'PHISAND': {
        'title': 'PHISAND',
        'titlefont': {'color': COLOR_DICT['PHISAND'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['PHISAND'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .89, 'title_standoff': .1,
        'dtick': 0.1, 'range': [0, 1], 'type': 'linear', 'zeroline': False, 'overlaying': 'x4'
    },
    'VSH_LAM': {
        'title': 'VSH_LAM',
        'titlefont': {'color': COLOR_DICT['VSH_LAM'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['VSH_LAM'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .89, 'title_standoff': .1,
        'dtick': 0.1, 'range': [0, 1], 'type': 'linear', 'zeroline': False, 'overlaying': 'x6'
    },
    'VSH_DIS': {
        'title': 'VSH_DIS',
        'titlefont': {'color': COLOR_DICT['VSH_DIS'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['VSH_DIS'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .93, 'title_standoff': .1,
        'dtick': 0.1, 'range': [0, 1], 'type': 'linear', 'zeroline': False, 'overlaying': 'x6'
    },
    'SW_TS': {
        'title': 'SW_TS',
        'titlefont': {'color': COLOR_DICT['SW_TS'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['SW_TS'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .89, 'title_standoff': .1,
        'dtick': 0.2, 'range': [0, 1], 'type': 'linear', 'zeroline': False, 'overlaying': 'x5'
    },
    'SWT': {
        'title': 'SWT',
        'titlefont': {'color': COLOR_DICT['SWT'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['SWT'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85, 'title_standoff': .1,
        'dtick': 0.2, 'range': [0, 1], 'type': 'linear', 'zeroline': False
    },
}

In [ ]:
from quick_pp.plotter.well_log import plotly_log

    # Plot the results
fig = plotly_log(filtered_df, depth_uom='ft', trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
